# **Graph Analytics Demo**
**Using Spark GraphX**



---


# **Spark and GraphFrames Set Up**


In [1]:
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.2.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

# install findspark using pip
!pip install -q findspark

# install pyspark
!pip3 install pyspark==3.2.0

# install graphframes
!pip3 install graphframes

zsh:1: command not found: apt-get
zsh:1: command not found: wget
tar: Error opening archive: Failed to open 'spark-3.2.0-bin-hadoop3.2.tgz'
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     |████████████████████████████████| 281.3 MB 71.2 MB/s eta 0:00:012
     |████████████████████████████████| 198 kB 103.3 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=e9f17ac07e469a3ec02b7feb699ba3bde19924dad1d25fb303f0aad7ebb1ac50
  Stored in directory: /Users/dhruvnarayan/Library/Caches/pip/wheels/2f/f8/95/2ad14a4614b4a9f645ee928fbbd057b1b254c67adb494c9a58
Successfully built pyspark
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you ar

Download the graphframes jar file from: [Graphframe jar file:](https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar)

Upload it in the Google Colab Files folder. Can be found in the left pane of this window.

In [2]:
!cp -v /content/graphframes-0.8.2-spark3.2-s_2.12.jar $SPARK_HOME/jars/

'/content/graphframes-0.8.2-spark3.2-s_2.12.jar' -> '/content/spark-3.2.0-bin-hadoop3.2/jars/graphframes-0.8.2-spark3.2-s_2.12.jar'


In [2]:
#import the packages
from pyspark import *
from pyspark.sql import *
from graphframes import *
import findspark
import pandas as pd

findspark.init()

# Start a Spark session
spark = SparkSession.builder.master("local[*]").getOrCreate()

ModuleNotFoundError: No module named 'pyspark'

In [58]:
from pyspark.context import SparkContext
sc = SparkContext.getOrCreate(SparkConf())
sc.setCheckpointDir('/content/sample_data')
# SparkContext.setCheckpointDir(dirName= '/content/sample_data')


# **CPDB Example**

Analyze the police officers connection with allegations

In [4]:
import psycopg2

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [5]:
# access the postgresql server
conn = psycopg2.connect(
    host="codd04.research.northwestern.edu",
    port = "5433",
    database="postgres",
    user="cpdbstudent",
    password="DataSci4AI")

In [6]:
cursor = conn.cursor()

**Following query creates nodes and edges to answer the questions.**



*   **nodes**: id, officer name and allegation count
*   **edges**: src(officer1 id), dist(officer2 id) and relationship(allegation count)

In [7]:
from google.colab import files
files.upload()

Saving officer_css_awards.csv to officer_css_awards.csv


{'officer_css_awards.csv': b'officer_id,css,award_count\n1,1.7993,88\n2,6.7814,100\n6,1.507,2\n7,2.9485,25\n8,0.5129,1\n9,0.6247,0\n11,1.6297,2\n13,2.8779,8\n14,1.2065,7\n15,1.0667,32\n16,15.2223,48\n17,5.8316,6\n18,1.4782,43\n19,0.755,14\n21,2.4225,0\n25,1.3714,5\n27,0.9164,0\n29,0.8294,22\n33,3.6575,2\n34,6.1552,19\n35,0.4372,9\n38,4.5348,45\n39,5.6589,30\n41,3.2107,25\n42,5.6047,177\n44,8.6581,26\n45,0.0877,2\n46,2.234,2\n47,2.4042,32\n48,2.2254,23\n49,4.4657,112\n50,0.064,13\n51,3.5778,48\n52,7.1977,63\n53,0.11,8\n54,4.0196,15\n56,10.3933,53\n58,9.7756,15\n59,3.7696,0\n60,15.0506,139\n61,3.8629,50\n62,1.6642,20\n64,3.105,60\n65,15.3788,98\n66,0.9713,6\n67,5.2396,49\n69,5.1378,22\n71,0.1853,1\n72,4.8183,11\n74,0.84,7\n75,11.0836,43\n77,2.0835,38\n80,8.7464,19\n81,5.78,52\n82,0.2253,3\n83,2.9704,16\n87,1.1262,16\n89,1.457,3\n92,9.0708,35\n93,3.0157,14\n94,7.1591,71\n96,1.7769,3\n100,0.9195,0\n103,0.4667,2\n104,2.234,2\n105,2.8623,7\n107,2.8471,11\n108,1.7264,17\n109,0.4176,6\n111,5.1

In [8]:
officer_css_awards = spark.read.csv('officer_css_awards.csv',inferSchema=True, header =True)
officer_css_awards.show()

+----------+-------+-----------+
|officer_id|    css|award_count|
+----------+-------+-----------+
|         1| 1.7993|         88|
|         2| 6.7814|        100|
|         6|  1.507|          2|
|         7| 2.9485|         25|
|         8| 0.5129|          1|
|         9| 0.6247|          0|
|        11| 1.6297|          2|
|        13| 2.8779|          8|
|        14| 1.2065|          7|
|        15| 1.0667|         32|
|        16|15.2223|         48|
|        17| 5.8316|          6|
|        18| 1.4782|         43|
|        19|  0.755|         14|
|        21| 2.4225|          0|
|        25| 1.3714|          5|
|        27| 0.9164|          0|
|        29| 0.8294|         22|
|        33| 3.6575|          2|
|        34| 6.1552|         19|
+----------+-------+-----------+
only showing top 20 rows



In [9]:
edges_query = "SELECT da1.officer_id src, da2.officer_id dst, COUNT(DISTINCT da1.allegation_id) relationship FROM data_officerallegation da1 JOIN data_officerallegation da2 ON da1.allegation_id = da2.allegation_id AND da1.officer_id < da2.officer_id GROUP BY da1.officer_id, da2.officer_id ORDER BY count(*) DESC;"

In [10]:
nodes_query = "SELECT  id, first_name || ' ' || last_name officer_name, allegation_count FROM data_officer;"

In [12]:
cursor.execute(edges_query)
edges = cursor.fetchall()
print("shape is: " + str(len(edges))) # 17465

df_edges = pd.DataFrame(edges)
colnames = [desc[0] for desc in cursor.description]
df_edges.columns = colnames

print(df_edges.shape)

shape is: 229195
(229195, 3)


In [13]:
df_edges.head()

,src,dst,relationship
0,12478,32166,53
1,8562,27778,47
2,1553,10724,43
3,2725,21703,41
4,3605,14442,41


In [14]:
cursor.execute(nodes_query)
nodes = cursor.fetchall()
print("shape is: " + str(len(nodes))) # 17465

df_nodes = pd.DataFrame(nodes)
colnames = [desc[0] for desc in cursor.description]
df_nodes.columns = colnames

print(df_nodes.shape)

shape is: 35545
(35545, 3)


In [15]:
df_nodes.head()

,id,officer_name,allegation_count
0,32312,Randall Ryan,10.0
1,32358,Kevin Stoll,24.0
2,32480,David Lavin,0.0
3,32491,Kevin Rake,0.0
4,2680,Richard Brandt,0.0


In [16]:
edges_ = spark.createDataFrame(df_edges)

In [17]:
nodes = spark.createDataFrame(df_nodes)

In [18]:
# Joining officer_css_awards with nodes and edges
officer_css_awards.createOrReplaceTempView('officer_css_awards')
edges_.createOrReplaceTempView('edges_')
nodes.createOrReplaceTempView('nodes')

nodes_joined = spark.sql("""
    SELECT A.*, B.CSS, B.award_count 
    FROM nodes A inner join officer_css_awards B on A.id = B.officer_id
""")

edges_joined = spark.sql("""
    SELECT A.*, B.CSS as src_CSS, B.award_count as src_award_count, C.CSS as dst_CSS, C.award_count as dst_award_count
    FROM edges_ A INNER JOIN officer_css_awards B on A.src = B.officer_id
    INNER JOIN officer_css_awards C ON A.dst = C.officer_id
""")

In [19]:
nodes_joined.show()

+-----+--------------------+----------------+------+-----------+
|   id|        officer_name|allegation_count|   CSS|award_count|
+-----+--------------------+----------------+------+-----------+
|32312|        Randall Ryan|            10.0|4.2757|         35|
|32358|         Kevin Stoll|            24.0|9.8512|        102|
|31804|      Diana Anderson|             1.0| 0.252|         13|
| 8175|    Christina Fabian|            11.0|2.3819|          6|
| 9534|      Gerardo Garcia|            10.0|1.5739|          8|
|25219|      Linda Salustro|             8.0|  1.52|          4|
|15636|          Marie Lane|             8.0|1.9359|          8|
| 3613|       Patricia Cain|             4.0|0.7031|          7|
|31793|      Adam Aleszczyk|             3.0|1.1367|         62|
| 4022|     Michael Carroll|             6.0|1.8722|         11|
|32005|          Tony Green|            11.0|4.0107|         17|
|32080|        Eric Johnson|             5.0| 1.323|         18|
|32188|     Elmore Metcal

In [20]:
edges_joined.show()

+-----+-----+------------+-------+---------------+-------+---------------+
|  src|  dst|relationship|src_CSS|src_award_count|dst_CSS|dst_award_count|
+-----+-----+------------+-------+---------------+-------+---------------+
|12478|32166|          53|  32.48|             22|41.0226|            167|
| 8562|27778|          47|52.7218|              1|29.2392|              9|
| 1553|10724|          43|29.4486|            113|28.4023|            155|
| 2725|21703|          41|29.4523|             73|21.3978|            103|
| 3605|14442|          41| 40.512|            182|  38.09|            212|
|12479|20713|          40|18.3478|             35| 16.934|             15|
| 8658|13788|          38|41.9424|             14|51.3158|              1|
|14045|15502|          37|11.3761|             14|13.6153|             28|
| 2369| 7015|          36| 17.355|             78|17.1266|              3|
| 1553|16699|          35|29.4486|            113| 18.111|            119|
|12074|12825|          35

In [22]:
cpdb = GraphFrame(nodes_joined, edges_joined)

In [24]:
cpdb.vertices.show()

+-----+--------------------+----------------+------+-----------+
|   id|        officer_name|allegation_count|   CSS|award_count|
+-----+--------------------+----------------+------+-----------+
|32312|        Randall Ryan|            10.0|4.2757|         35|
|32358|         Kevin Stoll|            24.0|9.8512|        102|
|31804|      Diana Anderson|             1.0| 0.252|         13|
| 8175|    Christina Fabian|            11.0|2.3819|          6|
| 9534|      Gerardo Garcia|            10.0|1.5739|          8|
|25219|      Linda Salustro|             8.0|  1.52|          4|
|15636|          Marie Lane|             8.0|1.9359|          8|
| 3613|       Patricia Cain|             4.0|0.7031|          7|
|31793|      Adam Aleszczyk|             3.0|1.1367|         62|
| 4022|     Michael Carroll|             6.0|1.8722|         11|
|32005|          Tony Green|            11.0|4.0107|         17|
|32080|        Eric Johnson|             5.0| 1.323|         18|
|32188|     Elmore Metcal

In [25]:
cpdb.edges.show()

+-----+-----+------------+-------+---------------+-------+---------------+
|  src|  dst|relationship|src_CSS|src_award_count|dst_CSS|dst_award_count|
+-----+-----+------------+-------+---------------+-------+---------------+
|12478|32166|          53|  32.48|             22|41.0226|            167|
| 8562|27778|          47|52.7218|              1|29.2392|              9|
| 1553|10724|          43|29.4486|            113|28.4023|            155|
| 2725|21703|          41|29.4523|             73|21.3978|            103|
| 3605|14442|          41| 40.512|            182|  38.09|            212|
|12479|20713|          40|18.3478|             35| 16.934|             15|
| 8658|13788|          38|41.9424|             14|51.3158|              1|
|14045|15502|          37|11.3761|             14|13.6153|             28|
| 2369| 7015|          36| 17.355|             78|17.1266|              3|
| 1553|16699|          35|29.4486|            113| 18.111|            119|
|12074|12825|          35

# This table shows you the distribution of CSS for all officers

In [34]:
cpdb.vertices.selectExpr('percentile_approx(CSS, 0.1)').show()
cpdb.vertices.selectExpr('percentile_approx(CSS, 0.25)').show()
cpdb.vertices.selectExpr('percentile_approx(CSS, 0.5)').show()
cpdb.vertices.selectExpr('percentile_approx(CSS, 0.75)').show()
cpdb.vertices.selectExpr('percentile_approx(CSS, 0.95)').show()
cpdb.vertices.selectExpr('percentile_approx(CSS, 0.99)').show()

+----------------------------------+
|percentile_approx(CSS, 0.1, 10000)|
+----------------------------------+
|                            0.4326|
+----------------------------------+

+-----------------------------------+
|percentile_approx(CSS, 0.25, 10000)|
+-----------------------------------+
|                             1.1363|
+-----------------------------------+

+----------------------------------+
|percentile_approx(CSS, 0.5, 10000)|
+----------------------------------+
|                            2.7191|
+----------------------------------+

+-----------------------------------+
|percentile_approx(CSS, 0.75, 10000)|
+-----------------------------------+
|                             5.7014|
+-----------------------------------+

+-----------------------------------+
|percentile_approx(CSS, 0.95, 10000)|
+-----------------------------------+
|                            13.7504|
+-----------------------------------+

+-----------------------------------+
|percentile_appro

# This section shows you the increasing trend in awards received by first degree connections as CSS increases

In [37]:
cpdb.edges.createOrReplaceTempView('cpdb_edges')
cpdb.vertices.createOrReplaceTempView('cpdb_vertices')

In [46]:
spark.sql("""
select avg(avg_first_degree_awards) as avg_first_degree_awards_10_pct
from (select avg(dst_award_count) as avg_first_degree_awards
from cpdb_edges
where src_CSS < (select percentile_approx(CSS, 0.1) from cpdb_vertices)
group by src)
""").show()

spark.sql("""
select avg(avg_first_degree_awards) as avg_first_degree_awards_10_to_20_pct
from (select avg(dst_award_count) as avg_first_degree_awards
from cpdb_edges
where src_CSS between (select percentile_approx(CSS, 0.1) from cpdb_vertices) and (select percentile_approx(CSS, 0.2) from cpdb_vertices)
group by src)
""").show()

spark.sql("""
select avg(avg_first_degree_awards) as avg_first_degree_awards_20_to_30_pct
from (select avg(dst_award_count) as avg_first_degree_awards
from cpdb_edges
where src_CSS between (select percentile_approx(CSS, 0.2) from cpdb_vertices) and (select percentile_approx(CSS, 0.3) from cpdb_vertices)
group by src)
""").show()

spark.sql("""
select avg(avg_first_degree_awards) as avg_first_degree_awards_30_to_40_pct
from (select avg(dst_award_count) as avg_first_degree_awards
from cpdb_edges
where src_CSS between (select percentile_approx(CSS, 0.3) from cpdb_vertices) and (select percentile_approx(CSS, 0.4) from cpdb_vertices)
group by src)
""").show()

spark.sql("""
select avg(avg_first_degree_awards) as avg_first_degree_awards_40_to_50_pct
from (select avg(dst_award_count) as avg_first_degree_awards
from cpdb_edges
where src_CSS between (select percentile_approx(CSS, 0.4) from cpdb_vertices) and (select percentile_approx(CSS, 0.5) from cpdb_vertices)
group by src)
""").show()

spark.sql("""
select avg(avg_first_degree_awards) as avg_first_degree_awards_50_to_60_pct
from (select avg(dst_award_count) as avg_first_degree_awards
from cpdb_edges
where src_CSS between (select percentile_approx(CSS, 0.5) from cpdb_vertices) and (select percentile_approx(CSS, 0.6) from cpdb_vertices)
group by src)
""").show()

spark.sql("""
select avg(avg_first_degree_awards) as avg_first_degree_awards_60_to_70_pct
from (select avg(dst_award_count) as avg_first_degree_awards
from cpdb_edges
where src_CSS between (select percentile_approx(CSS, 0.6) from cpdb_vertices) and (select percentile_approx(CSS, 0.7) from cpdb_vertices)
group by src)
""").show()

spark.sql("""
select avg(avg_first_degree_awards) as avg_first_degree_awards_70_to_80_pct
from (select avg(dst_award_count) as avg_first_degree_awards
from cpdb_edges
where src_CSS between (select percentile_approx(CSS, 0.7) from cpdb_vertices) and (select percentile_approx(CSS, 0.8) from cpdb_vertices)
group by src)
""").show()

spark.sql("""
select avg(avg_first_degree_awards) as avg_first_degree_awards_80_to_90_pct
from (select avg(dst_award_count) as avg_first_degree_awards
from cpdb_edges
where src_CSS between (select percentile_approx(CSS, 0.8) from cpdb_vertices) and (select percentile_approx(CSS, 0.9) from cpdb_vertices)
group by src)
""").show()

spark.sql("""
select avg(avg_first_degree_awards) as avg_first_degree_awards_90_to_95_pct
from (select avg(dst_award_count) as avg_first_degree_awards
from cpdb_edges
where src_CSS between (select percentile_approx(CSS, 0.9) from cpdb_vertices) and (select percentile_approx(CSS, 0.95) from cpdb_vertices)
group by src)
""").show()

spark.sql("""
select avg(avg_first_degree_awards) as avg_first_degree_awards_95_to_99_pct
from (select avg(dst_award_count) as avg_first_degree_awards
from cpdb_edges
where src_CSS between (select percentile_approx(CSS, 0.95) from cpdb_vertices) and (select percentile_approx(CSS, 0.99) from cpdb_vertices)
group by src)
""").show()

spark.sql("""
select avg(avg_first_degree_awards) as avg_first_degree_awards_99_above_pct
from (select avg(dst_award_count) as avg_first_degree_awards
from cpdb_edges
where src_CSS > (select percentile_approx(CSS, 0.99) from cpdb_vertices)
group by src)
""").show()

+------------------------------+
|avg_first_degree_awards_10_pct|
+------------------------------+
|            21.860159803955916|
+------------------------------+

+------------------------------------+
|avg_first_degree_awards_10_to_20_pct|
+------------------------------------+
|                   25.46627998993069|
+------------------------------------+

+------------------------------------+
|avg_first_degree_awards_20_to_30_pct|
+------------------------------------+
|                  30.047711027528198|
+------------------------------------+

+------------------------------------+
|avg_first_degree_awards_30_to_40_pct|
+------------------------------------+
|                   32.95962149742117|
+------------------------------------+

+------------------------------------+
|avg_first_degree_awards_40_to_50_pct|
+------------------------------------+
|                  36.791168060871605|
+------------------------------------+

+------------------------------------+
|avg_first_

**Implement Graph Algorithms**

In [26]:
tc_cpdb = cpdb.triangleCount()
tc_cpdb.show()

+-----+-----+--------------------+----------------+------+-----------+
|count|   id|        officer_name|allegation_count|   CSS|award_count|
+-----+-----+--------------------+----------------+------+-----------+
|   13|32113|       Kurt Kourakis|             7.0|2.7362|         24|
|  295|32188|     Elmore Metcalfe|            27.0| 8.619|         17|
|  120|32308|        Giselle Ruiz|             2.0|0.2719|         40|
|   11|32080|        Eric Johnson|             5.0| 1.323|         18|
|    1|27908|        Flora Suttle|             5.0|1.1471|          5|
|   26|32005|          Tony Green|            11.0|4.0107|         17|
|   11|15636|          Marie Lane|             8.0|1.9359|          8|
|    1| 9534|      Gerardo Garcia|            10.0|1.5739|          8|
|    0|31793|      Adam Aleszczyk|             3.0|1.1367|         62|
|    7| 2642|Leatheia Brady-Rh...|             3.0|0.7727|         14|
|   33| 2735|Charles Breckenridge|            10.0|2.7615|          6|
|    0

In [50]:
tc_cpdb.count()

20352

# This table shows you the high CSS scores for the most connected officers

### Just for reference: CSS 95th percentile is 13.75

In [48]:
pr_cpdb = cpdb.pageRank(resetProbability=0.15, tol=0.01)
#look at the pagerank score for every vertex
pr_cpdb.vertices.orderBy('pagerank', ascending=False).show()

+-----+------------------+----------------+-------+-----------+------------------+
|   id|      officer_name|allegation_count|    CSS|award_count|          pagerank|
+-----+------------------+----------------+-------+-----------+------------------+
|32442|      John Zinchuk|            23.0|12.8494|         97|122.85961890322304|
|32440|       Mark Zawila|            34.0|13.4627|         25|   82.596692708314|
|32425|    Perry Williams|            27.0|11.5465|        118|  72.0279045279787|
|32410|     Joseph Watson|            29.0|10.8425|         62| 68.47949785853883|
|32430|    Michael Wrobel|            22.0|16.7592|        165|  67.9187831349547|
|32350|    Robert Spiegel|            20.0|13.0244|        183| 65.39503775647411|
|32351|  Boonserm Srisuth|            25.0|11.0281|         75| 63.01146602153837|
|32433|     Kenneth Yakes|            29.0| 9.8227|         53|   62.051713039664|
|32284|         Mark Reno|            76.0|32.1284|        142|61.834556455615996|
|324

# This table shows you the low CSS scores for the least influential officers

In [49]:
pr_cpdb.vertices.orderBy('pagerank', ascending=True).show()

+-----+-----------------+----------------+------+-----------+-----------------+
|   id|     officer_name|allegation_count|   CSS|award_count|         pagerank|
+-----+-----------------+----------------+------+-----------+-----------------+
| 9427|  Robert Gallegos|             1.0|0.2248|          0|0.323512343251966|
|30702|   Bobby Williams|             1.0|0.1727|          0|0.323512343251966|
|  812|    Patrick Arens|            10.0|   1.0|          6|0.323512343251966|
| 4184|Edward Castellano|             1.0|0.3044|          1|0.323512343251966|
| 9534|   Gerardo Garcia|            10.0|1.5739|          8|0.323512343251966|
|16767|Gregory Luszowiak|             2.0|0.6833|         26|0.323512343251966|
|32434|       Steven Yee|             2.0|0.3359|         25|0.323512343251966|
|32263|   Jeffrey Pineda|             1.0|0.8333|         21|0.323512343251966|
|16144|        Don Lewis|             2.0|0.4546|          2|0.323512343251966|
|32448|     James Martin|             5.

# This table shows you the average CSS for each percentile of pagerank

## You can see an increasing CSS value for the most connected officers

In [57]:
pr_cpdb.vertices.createOrReplaceTempView('pr_cpdb_vertices')

spark.sql("""
select avg(avg_css) as avg_css_25_pct
from (select avg(CSS) as avg_css
from pr_cpdb_vertices
where pagerank < (select percentile_approx(pagerank, 0.25) from pr_cpdb_vertices)
group by id)
""").show()

spark.sql("""
select avg(avg_css) as avg_css_25_to_50_pct
from (select avg(CSS) as avg_css
from pr_cpdb_vertices
where pagerank between (select percentile_approx(pagerank, 0.25) from pr_cpdb_vertices) and (select percentile_approx(pagerank, 0.5) from pr_cpdb_vertices)
group by id)
""").show()

spark.sql("""
select avg(avg_css) as avg_css_50_to_75_pct
from (select avg(CSS) as avg_css
from pr_cpdb_vertices
where pagerank between (select percentile_approx(pagerank, 0.5) from pr_cpdb_vertices) and (select percentile_approx(pagerank, 0.75) from pr_cpdb_vertices)
group by id)
""").show()

spark.sql("""
select avg(avg_css) as avg_css_75_to_90_pct
from (select avg(CSS) as avg_css
from pr_cpdb_vertices
where pagerank between (select percentile_approx(pagerank, 0.75) from pr_cpdb_vertices) and (select percentile_approx(pagerank, 0.90) from pr_cpdb_vertices)
group by id)
""").show()

spark.sql("""
select avg(avg_css) as avg_css_90_to_95_pct
from (select avg(CSS) as avg_css
from pr_cpdb_vertices
where pagerank between (select percentile_approx(pagerank, 0.90) from pr_cpdb_vertices) and (select percentile_approx(pagerank, 0.95) from pr_cpdb_vertices)
group by id)
""").show()

spark.sql("""
select avg(avg_css) as avg_css_95_above_pct
from (select avg(CSS) as avg_css
from pr_cpdb_vertices
where pagerank > (select percentile_approx(pagerank, 0.95) from pr_cpdb_vertices)
group by id)
""").show()


+------------------+
|    avg_css_25_pct|
+------------------+
|2.0275589068029927|
+------------------+

+--------------------+
|avg_css_25_to_50_pct|
+--------------------+
|  3.7916747838050373|
+--------------------+

+--------------------+
|avg_css_50_to_75_pct|
+--------------------+
|    4.51714002751031|
+--------------------+

+--------------------+
|avg_css_75_to_90_pct|
+--------------------+
|   5.777709070072033|
+--------------------+

+--------------------+
|avg_css_90_to_95_pct|
+--------------------+
|   6.690802355250243|
+--------------------+

+--------------------+
|avg_css_95_above_pct|
+--------------------+
|  10.048070461236486|
+--------------------+

